In [1]:
from libs.data_manipulation import *
from libs.feature_generation import *
from libs.dim_reduction import *
from libs.football_plots import *
from libs.clustering import *
from libs.convex_hull import *
from libs.weight_generator import *
from scipy.stats import wasserstein_distance_nd


/home/martin/.local/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (None)/charset_normalizer (2.1.0) doesn't match a supported version!
  warnings.warn(


In [2]:
england_df = compile_team_tracking_data("data", "England")


In [21]:
def normalize_positions_with_ball(df):
    # Create copies of ball x and y columns for both home and away teams
    df_normalized = df.copy()
    
    # Normalize 'home' team player positions
    home_columns_x = [col for col in df.columns if col.startswith('home') and col.endswith('_x')]
    home_columns_y = [col for col in df.columns if col.startswith('home') and col.endswith('_y')]
    
    for x_col, y_col in zip(home_columns_x, home_columns_y):
        df_normalized[x_col] = df[x_col] - df['ball_x']
        df_normalized[y_col] = df[y_col] - df['ball_y']
    
    # Normalize 'away' team player positions
    away_columns_x = [col for col in df.columns if col.startswith('away') and col.endswith('_x')]
    away_columns_y = [col for col in df.columns if col.startswith('away') and col.endswith('_y')]
    
    for x_col, y_col in zip(away_columns_x, away_columns_y):
        df_normalized[x_col] = df[x_col] - df['ball_x']
        df_normalized[y_col] = df[y_col] - df['ball_y']
    
    return df_normalized

In [22]:
one_match = extract_one_match(england_df,3).iloc[::48]
one_match_oringal = one_match

identified_corner_timestamp = 4432.84

In [23]:



identified_corner_df = one_match[one_match["Time [s]"] == identified_corner_timestamp]

func = lambda x : 1/x


inverse_identified_corner_weights = calculate_weights(identified_corner_df, fun = func)
inverse_distance_list = calculate_weights(one_match, fun= func) #Inverse proportionality to distance

In [24]:
from scipy.stats import wasserstein_distance_nd
one_match = normalize_positions_with_ball(one_match)
coordinates = one_match.filter(regex ="^home").to_numpy()


identified_corner_coordinates_numpy = identified_corner_df.to_numpy()
identified_corner_coordinates = [list(zip(row[~np.isnan(row)][::2],row[~np.isnan(row)][1::2])) for row in coordinates]
coordinates_zipped = [list(zip(row[~np.isnan(row)][::2],row[~np.isnan(row)][1::2])) for row in coordinates]


In [25]:
print(len(inverse_identified_corner_weights[0]))


11


In [26]:

#Get closest situations
distances = []
i = 0
for weights, coordinates in zip(inverse_distance_list, coordinates_zipped):
    i+=1
    if(not np.isnan(np.sum(weights)) and len(weights) == 11 and len( coordinates) == 11):
        
        distances.append((wasserstein_distance_nd(identified_corner_coordinates[0], coordinates, u_weights= inverse_identified_corner_weights[0], v_weights=weights), i))

In [27]:
indices_and_distances = sorted(distances, key = lambda t: t[0])
indices = [index for _,index in indices_and_distances]
print(indices)

[1, 4697, 4696, 9481, 9763, 8956, 10044, 4121, 6188, 6432, 5054, 4698, 9971, 9414, 5055, 636, 8693, 9236, 8142, 9325, 9972, 3236, 9482, 8718, 9499, 9237, 9494, 1685, 9230, 3282, 7940, 4695, 2296, 5421, 9917, 4586, 32, 8719, 4070, 4080, 8992, 447, 4651, 138, 9492, 4113, 954, 3141, 6638, 619, 955, 133, 4699, 396, 8218, 4405, 9412, 9137, 8789, 5040, 9238, 5036, 9891, 9800, 868, 1938, 8963, 2253, 8141, 9339, 6435, 6482, 9405, 6281, 9488, 9487, 4079, 1165, 5056, 4069, 4404, 1440, 2899, 4820, 1991, 7402, 1514, 3345, 961, 726, 4686, 8030, 3975, 392, 9867, 4806, 1830, 5422, 9868, 5039, 9491, 459, 5181, 9244, 1595, 8358, 2744, 35, 4492, 7654, 6401, 1939, 9141, 635, 9537, 8632, 229, 4110, 8993, 7939, 727, 9326, 8633, 3552, 9808, 3337, 6275, 7752, 9324, 5516, 6402, 458, 5896, 4700, 463, 6075, 7329, 3235, 9235, 168, 9138, 8853, 747, 1947, 2420, 7655, 745, 6634, 3698, 8631, 8643, 4543, 9404, 1983, 9941, 3346, 9662, 2150, 4803, 8698, 9232, 6433, 230, 4452, 9586, 620, 1491, 3956, 1430, 4493, 9769, 80

In [28]:
situations = one_match.iloc[indices[:10]].index.to_numpy()


In [29]:

generate_pitches_from_start_indices(np.concatenate((situations,identified_corner_df.index.to_numpy())), one_match_oringal, "Ripleysk_k_plots",1,48)

/home/martin/anaconda3/envs/DBU/lib/python3.10/site-packages/matplotlib/quiver.py:640: RuntimeWarning: Mean of empty slice.
  amean = a[~self.Umask].mean()
/home/martin/.local/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/martin/anaconda3/envs/DBU/lib/python3.10/site-packages/matplotlib/quiver.py:640: RuntimeWarning: Mean of empty slice.
  amean = a[~self.Umask].mean()
/home/martin/.local/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
